In [5]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일에서 환경 변수 불러오기
load_dotenv()

endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME")
search_endpoint = os.getenv("SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_KEY")
search_index = os.getenv("SEARCH_INDEX_NAME")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

# 키 기반 인증을 사용하여 Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

prompt = "마지 여행사에서 제공하는 여행지는 어떤게 있어?"

chat_prompt = [
    {
        "role": "system",
        "content": "너는 마지여행사의 여행전문가야."
    },
    {
        "role": "user",
        "content": prompt
    }
]

completion = client.chat.completions.create(
    model=deployment,
    messages=chat_prompt,
    max_tokens=800,
    temperature=0.8,
    top_p=0.95,
    extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": search_endpoint,
                "index_name": search_index,
                "semantic_configuration": "1sesac000-margeis-index-semantic-configuration",
                "query_type": "semantic",
                "strictness": 2,
                "top_n_documents": 5,
                "authentication": {
                    "type": "api_key",
                    "key": search_key
                }
            }
        }]
    }
)

response = completion
print("Response: " + response.choices[0].message.content)


Response: 마지 여행사에서는 다양한 여행지를 제공하고 있습니다. 다음은 주요 여행지 목록입니다:

1. **두바이 (Dubai)**: 두바이는 현대적인 건축물과 역사적인 명소가 조화를 이루는 도시로, 다양한 관광 명소가 있습니다 [doc2].

2. **샌프란시스코 (San Francisco)**: 샌프란시스코는 독특한 문화와 아름다운 경관이 매력적인 도시입니다. 관광지로는 금문교와 알카트라즈 섬 등이 있습니다 [doc3].

3. **라스베이거스 (Las Vegas)**: 라스베이거스는 세계적으로 유명한 엔터테인먼트와 카지노의 중심지로, 다양한 공연과 액티비티가 제공됩니다 [doc4].

4. **뉴욕 (New York)**: 뉴욕은 다양한 문화와 관광 명소가 있는 도시로, 타임스퀘어와 센트럴파크 등 유명한 명소가 있습니다 [doc5].

이 외에도 마지 여행사는 다양한 여행지를 소개하고 있으며, 고객의 요구에 맞춘 맞춤형 여행 패키지도 제공합니다 .


In [ ]:
citations = response.choices[0].message.context['citations']
citations

In [ ]:
for c in citations:
    print(c['title'])